# Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()



In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[(crypto_df["IsTrading"] == True)]
crypto_df

In [ ]:
# Remove the "IsTrading" column. 
crypto_df_traded = crypto_df.drop(columns={"IsTrading"})
crypto_df_traded

In [ ]:
# Remove rows that have at least 1 null value.
crypto_df_traded = crypto_df_traded.dropna()
crypto_df_traded

In [ ]:
# Keep the rows where coins are mined.
crypto_df_traded = crypto_df_traded.loc[(crypto_df_traded["TotalCoinsMined"] > 0)]
crypto_df_traded


In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_df_names =pd.DataFrame(data=crypto_df_traded,columns =["CoinName","Unnamed: 0"])
crypto_df_names = crypto_df_names.set_index(["Unnamed: 0"])


In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df_traded = crypto_df_traded.drop(columns={"CoinName"})
crypto_df_traded = crypto_df_traded.set_index(["Unnamed: 0"])

In [ ]:

crypto_df_traded.head(10)


In [ ]:
# Use get_dummies() to create variables for text features.
X = crypto_df_traded.copy()
#X = X.drop("loan_status",axis=1)
X = pd.get_dummies(X, columns=["Algorithm","ProofType"])
X.head(10)

In [ ]:
# Standardize the data with StandardScaler().
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
X_scaled = data_scaler.fit_transform(X)
print(X_scaled)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [ ]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X_scaled)
pcs_df = pd.DataFrame(data = X_pca, columns=["PC1","PC2","PC3"], index=crypto_df_traded.index)

In [ ]:
pcs_df

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [28]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\Owner\conda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [31]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [32]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df Daasrames on the same columns.
clustered_df = pd.concat([crypto_df_traded, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(crypto_df_names, how='outer')

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.332154,1.070108,-0.596003,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315501,1.070105,-0.596402,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.301347,1.639901,-0.639441,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144164,-1.310302,0.188517,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.152688,-2.111651,0.400570,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.161463,-1.077301,-0.028540,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.393375,1.161502,-0.511605,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.146084,-2.139181,0.414294,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.151132,-2.111764,0.400554,Ethereum Classic,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [42]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color = "Class",
    symbol = "Class",
    width=800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"]
)
fig.show()

In [43]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'], sortable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [47]:
# Print the total number of tradable cryptocurrencies.
clustered_df['CoinName'].count()

532

In [69]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()
tcm_df = clustered_df[['TotalCoinsMined']]
tcs_df = clustered_df[['TotalCoinSupply']]
tcm_df_scaled = scaler.fit_transform(tcm_df)
tcs_df_scaled = scaler.fit_transform(tcs_df)

In [91]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(data=tcm_df_scaled, columns = ['TotalCoinsMined'])
scaled_df['TotalCoinSupply'] = tcs_df_scaled
scaled_df.index = clustered_df.index
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df = pd.concat([scaled_df,clustered_df['CoinName']], axis=1)
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df = pd.concat([scaled_df,clustered_df['Class']], axis=1)

#plot_df.head(10)

In [92]:
scaled_df

,TotalCoinsMined,TotalCoinSupply,CoinName,Class
Unnamed: 0,,,,
42,0.000000e+00,4.200000e-11,42 Coin,0
404,1.065855e-03,5.320000e-04,404Coin,0
1337,2.957551e-02,3.141593e-01,EliteCoin,0
BTC,1.810842e-05,2.100000e-05,Bitcoin,1
ETH,1.087731e-04,0.000000e+00,Ethereum,1
...,...,...,...,...
ZEPH,2.020225e-03,2.000000e-03,ZEPHYR,0
GAP,1.508199e-05,2.500000e-04,Gapcoin,0
BDX,9.901351e-04,1.400223e-03,Beldex,1


In [94]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)